# QCoDeS Example with Tektronix AWG70002A

The Tektronix awg70002A can operate in two modes: function generator mode or AWG mode. This example notebook briefly covers both.

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np
from qcodes.instrument_drivers.tektronix.AWG70002A import AWG70002A

In [2]:
awg = AWG70002A('awg', 'TCPIP0::172.20.2.101::inst0::INSTR', terminator='\n')

Connected to: TEKTRONIX AWG70002A (serial:B020397, firmware:FV:5.3.0128.0) in 0.08s


In [3]:
# Let's have a look at the available parameters

awg.print_readable_snapshot(update=True)

awg:
	parameter        value
--------------------------------------------------------------------------------
IDN               :	{'model': 'AWG70002A', 'vendor': 'TEKTRONIX', 'serial': '...
current_directory :	"\Users\OEM\Documents" 
mode              :	AWG 
timeout           :	10 (s)
awg_ch1:
	parameter      value
--------------------------------------------------------------------------------
fgen_amplitude  :	0.074 (V)
fgen_dclevel    :	0 (V)
fgen_frequency  :	1e+07 (Hz)
fgen_offset     :	0.12 (V)
fgen_period     :	1e-07 (s)
fgen_phase      :	25 (degrees)
fgen_signalpath :	direct 
fgen_symmetry   :	50 (%)
fgen_type       :	EXPONENTIALDECAY 
resolution      :	8 
state           :	0 
awg_ch2:
	parameter      value
--------------------------------------------------------------------------------
fgen_amplitude  :	0.5 (V)
fgen_dclevel    :	0 (V)
fgen_frequency  :	1e+07 (Hz)
fgen_offset     :	0 (V)
fgen_period     :	1e-07 (s)
fgen_phase      :	0 (degrees)
fgen_signalpath :	direct 
fgen_s

## Function Generator Style Operation

In [4]:
# Set the intrument mode to function generator
awg.mode('FGEN')

# Build some signal
awg.ch1.fgen_type('EXPONENTIALDECAY')
awg.ch1.fgen_frequency(10e6)
awg.ch1.fgen_amplitude(0.074)
awg.ch1.fgen_offset(0.12)
awg.ch1.fgen_phase(25)

# Switch channel 1 on
awg.ch1.state(1)

# Start outputting...
awg.play()

In [5]:
# switch off the output eventually
awg.stop()

# and disable the channel
awg.ch1.state(0)

## AWG Style Operation

The instrument can be operated as an awg where the user uploads arrays describing the waveforms.

Each channel operates in one of three resolution modes:

* 8 bit signal + 2 markers
* 9 bit signal + 1 marker
* 10 bit signal with no markers

Waveforms can be sent to the waveform list via `.wfmx` files. A `.wfmx` file can contain marker data. If the resolution of the instrument does not allow for markers, these are simply ignored.

### Making and sending waveforms to the waveform list

In [6]:
# set the instrument in awg mode
awg.mode('AWG')
# set the resolution to 8 bits plus two markers
awg.ch1.resolution(8)

In [7]:
# clear the sequence list and waveform list (NOT ALWAYS A GOOD IDEA! BE CAREFUL!)
awg.clearSequenceList()
awg.clearWaveformList()

In [8]:
# Let us make a sine, upload it and play it

N = 50000  # minimal length allowed is 2400 points

mysine = np.sin(10*2*np.pi*np.linspace(0, 1, N))
m1 = np.concatenate((np.ones(int(N/2)), np.zeros(int(N/2))))
m2 = np.concatenate((np.zeros(int(N/2)), np.zeros(int(N/2))))

data = np.array([mysine, m1, m2])

In [9]:
# The .wfmx file needs a name in the memory of the instrument
# The name of the waveform in the waveform list is that same name
# with no .wfmx extension
filename = 'examplewaveform.wfmx'

In [10]:
# now compile the binary file
wfmx_file = awg.makeWFMXFile(data)

In [11]:
# and send it and load it into memory
awg.sendWFMXFile(wfmx_file, filename)
awg.loadWFMXFile(filename)

In [12]:
# The waveform is now in the waveform list
awg.waveformList

['examplewaveform']

In [13]:
# now assign it to channel 1
awg.ch1.setWaveform(filename.replace('.wfmx', ''))

In [14]:
# Switch channel 1 on
awg.ch1.state(1)

# Start outputting...
awg.play()

In [15]:
# switch off the output eventually
awg.stop()

# and disable the channel
awg.ch1.state(0)

In [16]:
# Finally irreversibly tear down the instrument
awg.close()